In [ ]:
# Install required libraries
!pip install datasets

# Clone the repository
!git clone https://github.com/hausanlp/HERDPhobia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
# Import of important library
import pandas as pd
import torch
import os
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification, DataCollatorWithPadding
import numpy as np
from huggingface_hub import notebook_login
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
)
from datasets import Dataset, DatasetDict
import pandas as pd
import os
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load and preprocess data
train_df = pd.read_csv("HERDPhobia/train.tsv", sep="\t")
dev_df = pd.read_csv("HERDPhobia/dev.tsv", sep="\t")
test_df = pd.read_csv("HERDPhobia/test.tsv", sep="\t")

# Encode labels (if they're strings)
le = LabelEncoder()
train_df["label"] = le.fit_transform(train_df["label"])
dev_df["label"] = le.transform(dev_df["label"])
test_df["label"] = le.transform(test_df["label"])

# Convert to Hugging Face datasets
herdphobia_dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(dev_df),
    "test": Dataset.from_pandas(test_df),
})

# Load model and tokenizer
model_name = "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = herdphobia_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Define metrics function for multi-class classification
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted') # Changed to 'weighted'
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/404 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3090 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

Map:   0%|          | 0/884 [00:00<?, ? examples/s]

In [ ]:
# Check if there's class imbalance
train_label_counts = train_df['label'].value_counts()
print("Label distribution:", train_label_counts)

# Calculate class weights if needed

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
print("Class weights:", class_weights)

# Apply class weights in the model
# Note: You would typically do this inside the model's forward pass
# But with Hugging Face, we need a custom training loop or a custom model
# For simplicity, we'll use weighted data augmentation instead

# Weighted data augmentation for minority classes
def create_balanced_augmented_dataset(df, class_counts):
    # Find the majority class count
    max_count = class_counts.max()

    # Create dataframes for each class
    class_dfs = {}
    for class_label in class_counts.index:
        class_dfs[class_label] = df[df['label'] == class_label]

    # Augment each class to reach similar size as majority class
    augmented_dfs = []
    for class_label, class_df in class_dfs.items():
        if len(class_df) < max_count:
            # Calculate how many times to duplicate
            augment_factor = int(np.ceil(max_count / len(class_df))) - 1

            # Apply augmentation
            augmented_class_df = class_df.copy()
            for _ in range(augment_factor):
                # Apply text augmentation to the minority class
                augmented_texts = []
                augmented_labels = []

                for _, row in class_df.iterrows():
                    # Apply simple augmentation (you can use the techniques defined earlier)
                    # For now, just duplicate the example
                    augmented_texts.append(row['text'])
                    augmented_labels.append(row['label'])

                temp_df = pd.DataFrame({
                    'text': augmented_texts,
                    'label': augmented_labels
                })

                augmented_class_df = pd.concat([augmented_class_df, temp_df], ignore_index=True)

            # Take only what we need to balance
            augmented_class_df = augmented_class_df.sample(max_count, replace=False)
            augmented_dfs.append(augmented_class_df)
        else:
            augmented_dfs.append(class_df)

    # Combine all balanced classes
    balanced_df = pd.concat(augmented_dfs, ignore_index=True)
    return balanced_df

# Create a more balanced dataset
balanced_train_df = create_balanced_augmented_dataset(train_df, train_label_counts)
print(f"Original training set size: {len(train_df)}")
print(f"Balanced training set size: {len(balanced_train_df)}")
print("New class distribution:", balanced_train_df['label'].value_counts())

# Convert to Hugging Face dataset
balanced_train_dataset = Dataset.from_pandas(balanced_train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)
balanced_dataset_dict = DatasetDict({
    'train': balanced_train_dataset,
    'validation': dev_dataset,
    'test': test_dataset
})

# Tokenize the balanced dataset
tokenized_balanced_datasets = balanced_dataset_dict.map(tokenize_function, batched=True)
tokenized_balanced_datasets = tokenized_balanced_datasets.remove_columns(["text"])
tokenized_balanced_datasets = tokenized_balanced_datasets.rename_column("label", "labels")
tokenized_balanced_datasets.set_format("torch")

Label distribution: label
0    2466
1     624
Name: count, dtype: int64
Class weights: [0.62652068 2.47596154]
Original training set size: 3090
Balanced training set size: 4932
New class distribution: label
0    2466
1    2466
Name: count, dtype: int64


Map:   0%|          | 0/4932 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

Map:   0%|          | 0/884 [00:00<?, ? examples/s]

In [ ]:
# Set up improved training arguments
improved_training_args = TrainingArguments(
    output_dir="./improved_results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,  # Mixed precision training
    warmup_ratio=0.1,
    push_to_hub=False,
    logging_steps=50,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize improved trainer with balanced dataset
improved_trainer = Trainer(
    model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2),
    args=improved_training_args,
    train_dataset=tokenized_balanced_datasets["train"],
    eval_dataset=tokenized_balanced_datasets["validation"],
    compute_metrics=compute_metrics,
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Train improved model
improved_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohamed-pagna (mohamed-pagna-aims-cameroon) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.423700,0.430670,0.798186,0.806324,0.819032,0.798186
2,0.274700,0.693382,0.850340,0.849043,0.847924,0.850340
3,0.084000,1.231847,0.834467,0.831104,0.828622,0.834467
4,0.005100,1.411302,0.825397,0.823489,0.821840,0.825397
5,0.000100,1.560202,0.829932,0.823881,0.820313,0.829932


TrainOutput(global_step=3085, training_loss=0.2056677458641788, metrics={'train_runtime': 1906.1101, 'train_samples_per_second': 12.937, 'train_steps_per_second': 1.618, 'total_flos': 5745356854917120.0, 'train_loss': 0.2056677458641788, 'epoch': 5.0})

In [ ]:
# Evaluate improved model
improved_test_results = improved_trainer.evaluate(tokenized_balanced_datasets["test"])

# Print all available metrics first to verify keys
print("All available metrics:")
for key, value in improved_test_results.items():
    print(f"{key}: {value:.4f}")

# Binary classification report
print("\nBinary Classification Report:")
print(f"Accuracy: {improved_test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {improved_test_results['eval_f1']:.4f}")
print(f"Precision: {improved_test_results['eval_precision']:.4f}")
print(f"Recall: {improved_test_results['eval_recall']:.4f}")

# If you have per-class metrics (common format)
if 'eval_class_0_f1' in improved_test_results:
    print("\nPer-class Metrics:")
    print(f"Class 0 (Negative) F1: {improved_test_results['eval_class_0_f1']:.4f}")
    print(f"Class 1 (Positive) F1: {improved_test_results['eval_class_1_f1']:.4f}")
elif 'eval_f1_class0' in improved_test_results:  # Alternative naming convention
    print("\nPer-class Metrics:")
    print(f"Class 0 (Negative) F1: {improved_test_results['eval_f1_class0']:.4f}")
    print(f"Class 1 (Positive) F1: {improved_test_results['eval_f1_class1']:.4f}")

All available metrics:
eval_loss: 0.8619
eval_accuracy: 0.8258
eval_f1: 0.8214
eval_precision: 0.8183
eval_recall: 0.8258
eval_runtime: 4.9427
eval_samples_per_second: 178.8500
eval_steps_per_second: 11.3300
epoch: 5.0000

Binary Classification Report:
Accuracy: 0.8258
F1 Score: 0.8214
Precision: 0.8183
Recall: 0.8258


In [ ]:
from sklearn.metrics import classification_report

predictions = improved_trainer.predict(tokenized_balanced_datasets["test"])
print(classification_report(predictions.label_ids, predictions.predictions.argmax(-1)))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       705
           1       0.58      0.51      0.54       179

    accuracy                           0.83       884
   macro avg       0.73      0.71      0.72       884
weighted avg       0.82      0.83      0.82       884

